## 035. DEMO alpaca-lora 使用專用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:   Image_S_home_alpaca-lora_latest</span>.


### TRY IT
1. Lauch  a new notebook tab
2. Change to kernel:  Image_S_home_pytorch_2.1.0-cuda11.8-cudnn8-devel
3. In first cell add content below

```
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
```

In [ ]:
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [ ]:
!mkdir ~/github
%cd ~/github
!cp -rf /work/u00cjz00/slurm_jobs/demo/alpaca-lora .
%cd ~/github/alpaca-lora

In [ ]:
pip install fire intel_extension_for_pytorch -q

In [ ]:
### 資料集

In [ ]:
import pandas as pd
df = pd. read_json ( '/work/u00cjz00/slurm_jobs/github/dataset/school_math_30000.json' )
df

In [ ]:
%cd ~/github/alpaca-lora
!python3 finetune.py --help

### 2. 模型微調範例(一) 於 jupyter notebook 互動執行

In [ ]:
!wandb offline

In [ ]:
%cd ~/github/alpaca-lora
!wandb offline
!python3 finetune.py \
    --base_model '/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf' \
    --data_path '/work/u00cjz00/slurm_jobs/github/dataset/school_math_30000.json' \
    --output_dir './lora-alpaca' \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 3 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length

### 3. 模型微調範例(二) 於 jupyter notebook 使用 Singularity 執行

In [1]:
%%bash
cat << \EOF > lora.sh
The current working directory is: $PWD
You are logged in as $(whoami)

## 帳號套件下載
container_dir=${HOME}/container_demo/lora
home_dir=${container_dir}/home
tmp_dir=${container_dir}/tmp
output_dir=${container_dir}/output
mkdir -p ${home_dir} ${tmp_dir} ${output_dir}
rsync -avHS /work/u00cjz00/slurm_jobs/demo/alpaca-lora/ ${container_dir}/app

# 映像檔
IMAGE="/work/u00cjz00/nvidia/cjz_images/pytorch_2.1.0-cuda11.8-cudnn8-runtime_textgen.sif"

# 模型
MODEL_ID="/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf"

# 訓練資料及輸出結果目錄
input_data="/work/u00cjz00/slurm_jobs/github/dataset/school_math_30000.json";
output_data="${output_dir}/lora_school_math_30000";


# 安裝套件
singularity exec --nv -B /work -B ${container_dir}/home:${HOME} -B ${container_dir}/tmp:/tmp -B ${container_dir}/app:/app ${IMAGE} bash -c "cd /app;  pip install install fire intel_extension_for_pytorch -q"

# 執行訓練
singularity exec --nv -B /work -B ${container_dir}/home:${HOME} -B ${container_dir}/tmp:/tmp -B ${container_dir}/app:/app ${IMAGE} bash -c "cd /app;  python3 finetune.py --base_model \'${MODEL_ID}\' --data_path \'${input_data}\' \'${output_data}\'"

EOF

In [2]:
!cat lora.sh
#!bash lora.sh

The current working directory is: $PWD
You are logged in as $(whoami)

## 帳號套件下載
container_dir=${HOME}/container_demo/lora
home_dir=${container_dir}/home
tmp_dir=${container_dir}/tmp
output_dir=${container_dir}/output
mkdir -p ${home_dir} ${tmp_dir} ${output_dir}
rsync -avHS /work/u00cjz00/slurm_jobs/demo/alpaca-lora/ ${container_dir}/app

# 映像檔
IMAGE="/work/u00cjz00/nvidia/cjz_images/pytorch_2.1.0-cuda11.8-cudnn8-runtime_textgen.sif"

# 模型
MODEL_ID="/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf"

# 訓練資料及輸出結果目錄
input_data="/work/u00cjz00/slurm_jobs/github/dataset/school_math_30000.json";
output_data="${output_dir}/lora_school_math_30000";


# 安裝套件
singularity exec --nv -B /work -B ${container_dir}/home:${HOME} -B ${container_dir}/tmp:/tmp -B ${container_dir}/app:/app ${IMAGE} bash -c "cd /app;  pip install install fire intel_extension_for_pytorch -q"

# 執行訓練
singularity exec --nv -B /work -B ${container_dir}/home:${HOME} -B ${container_dir}/tmp:/tmp -B ${container_dir}/app

### 4. 模型微調範例(三) 於 jupyter notebook 使用 Slurm+ Singularity 執行

In [3]:
%%bash
cat << \EOF > lora.slurm
#!/work/u00cjz00/binary/bash5.0/bin/bash
#SBATCH -A GOV109189                                                    ### project number, Example GOV109189
#SBATCH -J _t2lora_                                                     ### Job name, Exmaple jupyterlab
#SBATCH -p gp4d                                                         ### Partition Name, Example ngs1gpu
#SBATCH --nodes=1                                                       ### Nodes, Default 1, node number
#SBATCH --ntasks-per-node=1                                             ### Tasks, Default 1, per node tasks
#SBATCH -c 4                                                            ### Cores assigned to each task, Example 4
#SBATCH --gres=gpu:1                                                    ### GPU number, Example gpu:1
##SBATCH --time=0-1:00:00                                               ### 停用 Runnung time, days-hours:minutes:seconds or hours:minutes:seconds
#SBATCH -o genai_%j.out                                                 ### Log folder, Here %j is job ID
#SBATCH -e genai_%j.err                                                 ### Log folder, Here %j is job ID


## 帳號套件下載
container_dir=${HOME}/container_demo/lora
home_dir=${container_dir}/home
tmp_dir=${container_dir}/tmp
output_dir=${container_dir}/output
mkdir -p ${home_dir} ${tmp_dir} ${output_dir}
rsync -avHS /work/g00cjz00/tmp_home/alpaca-lora_v3/ ${container_dir}/app

# 映像檔
IMAGE="/work/u00cjz00/nvidia/cjz_images/pytorch_2.1.0-cuda11.8-cudnn8-runtime_textgen.sif"

# 模型
MODEL_ID="/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf"

# 訓練資料及輸出結果目錄
input_data="/work/u00cjz00/slurm_jobs/github/dataset/school_math_30000.json";
output_data="${output_dir}/lora_school_math_30000";


# 安裝套件
ml libs/singularity/3.10.2
singularity exec --nv -B /work -B ${container_dir}/home:${HOME} -B ${container_dir}/tmp:/tmp -B ${container_dir}/app:/app ${IMAGE} bash -c "cd /app;  pip install install fire intel_extension_for_pytorch -q"

# 執行訓練
singularity exec --nv -B /work -B ${container_dir}/home:${HOME} -B ${container_dir}/tmp:/tmp -B ${container_dir}/app:/app ${IMAGE} bash -c "cd /app;  python3 finetune.py --base_model \'${MODEL_ID}\' --data_path \'${input_data}\' \'${output_data}\'" &

## Get pid and waitting for it
pid=$!
wait $pid

EOF

In [4]:
!cat  lora.slurm
#!sbatch lora.slurm

#!/work/u00cjz00/binary/bash5.0/bin/bash
#SBATCH -A GOV109189                                                    ### project number, Example GOV109189
#SBATCH -J _t2lora_                                                     ### Job name, Exmaple jupyterlab
#SBATCH -p gp4d                                                         ### Partition Name, Example ngs1gpu
#SBATCH --nodes=1                                                       ### Nodes, Default 1, node number
#SBATCH --ntasks-per-node=1                                             ### Tasks, Default 1, per node tasks
#SBATCH -c 4                                                            ### Cores assigned to each task, Example 4
#SBATCH --gres=gpu:1                                                    ### GPU number, Example gpu:1
##SBATCH --time=0-1:00:00                                               ### 停用 Runnung time, days-hours:minutes:seconds or hours:minutes:seconds
#SBATCH -o genai_%j.out                                    